## Goal: write python script to download daily weather data from wunderground for US airport stations, for last 30 years or so
- write function to download a single station and year
- to clean and flag data (download raw first, then make cleaned files?)
- deal with download errors
- * write a dataframe that says which stations/years are missing?
- write all data to a sqlite database

In [1]:
# This script downloads daily historical weather data from wunderground,
# for citibike analysis project.

# Saves a modified file to sqlite3 database

import sqlite3
import pandas as pd
import boto3


def get_wea_data_yearly(year,station):
    """
    Get historical (daily) weather data from wunderground for specified year and station
    """
    url = 'http://www.wunderground.com/history/airport/' + station + '/' + str(year) + '/1/1/CustomHistory.html?dayend=31&monthend=12&yearend=' + str(year) + '&req_city=NA&req_state=NA&req_statename=NA&format=1'
    dat = pd.read_csv(url,parse_dates=True)
    # Name of date column is tz, which varies so we can't hardwire name
    dat.iloc[:,0] =  pd.to_datetime(dat.iloc[:,0])
    dat['date'] = dat.iloc[:,0]
    dat.set_index(dat.date, inplace=True)
    dat['yday']  = dat.date.dt.dayofyear
    dat['month'] = dat.date.dt.month
    dat['year']  = dat.date.dt.year
    dat.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
    dat['st_code'] = station
    vars_to_keep = ['date','st_code','Max_TemperatureF','Min_TemperatureF','Mean_TemperatureF','year','yday','month','PrecipitationIn','_CloudCover','_Max_Gust_SpeedMPH','_Events']
    dat = dat[vars_to_keep]
    dat.rename(columns={'Max_TemperatureF':'max_temp','Min_TemperatureF':'min_temp','Mean_TemperatureF':'mean_temp','PrecipitationIn':'precip_In','_CloudCover':
        'cloud_cover','_Max_Gust_SpeedMPH':'max_gust_mph','_Events':'events'},inplace=True)    
    
    return dat



In [2]:
# test function
dat = get_wea_data_yearly(2000,'KDEN')
dat.head()
dat.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 271 entries, 2000-01-01 to 2000-12-31
Data columns (total 12 columns):
date            271 non-null datetime64[ns]
st_code         271 non-null object
max_temp        271 non-null int64
min_temp        271 non-null int64
mean_temp       269 non-null float64
year            271 non-null int64
yday            271 non-null int64
month           271 non-null int64
precip_In       271 non-null float64
cloud_cover     271 non-null int64
max_gust_mph    152 non-null float64
events          82 non-null object
dtypes: datetime64[ns](1), float64(3), int64(6), object(2)
memory usage: 27.5+ KB


In [3]:
sta_list = pd.read_csv('USAirportWeatherStations.csv')
sta_list.head()

,Unnamed: 0,Station,State,airportCode,Lat,Lon,Elevation,WMO
0,1,Central,AK,PARL,65.57,-144.80,292,99999
1,2,Atka,AK,PAAK,52.22,-174.20,17,99999
2,3,Buckland,AK,PABL,65.99,-161.12,0,99999
3,4,Portage Glacier,AK,PATO,60.79,-148.83,29,99999
4,5,Nivalina,AK,PAVL,67.73,-164.55,3,70148


In [8]:
#sta_list[sta_list.airportCode=="PAVW"]
stcodes = sta_list.airportCode.values
stcodes[106]
stcodes = stcodes[106:]
type(stcodes)
stcodes[0:2]
#len(stcodes)

array(['PAVW', 'PASK'], dtype=object)

In [ ]:
# Write to databse

# start at PAVW

con = sqlite3.connect('wunderground_daily.db')
years = range(1950,2017)
for sta in stcodes:
    print('getting data for ' + sta )
    for year in years:
        #print('getting data for ' + sta + ' for ' + str(year))
        try:
            dat = get_wea_data_yearly(year,sta)
            if (dat.shape[0]!=0):
                dat.to_sql("wea",con,if_exists='append',index=False)
                # write to S3 also
                s3 = boto3.resource('s3')
                #fname = csv_name
                key_name = sta + '_' + str(year) + '.csv'
                #data = open(fname, 'rb')
                data = dat.to_csv(None,index=False)
                s3.Bucket('wundergrounddaily').put_object(Key=key_name, Body=data)
        except:
            pass

            del dat


getting data for PAVW
getting data for PASK
getting data for PAWG
getting data for PAWS
getting data for PAEC
getting data for PAUO
getting data for PAHN
getting data for PABN
getting data for PAJV
getting data for PAEI
getting data for PATC
getting data for PATL
getting data for PAEM
getting data for PAKW
getting data for PAGS
getting data for PAMX
getting data for PASA
getting data for PAOH
getting data for PAIM
getting data for PAQT
getting data for KEET
getting data for KANB
getting data for KAUO
getting data for KBFM
getting data for KBHM
getting data for KCKL
getting data for KDCU
getting data for KDHN
getting data for KGZH
getting data for KGAD
getting data for KHSV
getting data for KMGM
getting data for KMOB
getting data for KMSL
getting data for KMXF
getting data for KOZR
getting data for KTCL
getting data for KTOI
getting data for KSRC
getting data for KXNA
getting data for KMWT
getting data for KRUE
getting data for KARG
getting data for KASG
getting data for KBVX
getting da

getting data for KDEH
getting data for KDNS
getting data for KDSM
getting data for KDVN
getting data for KEBS
getting data for KEOK
getting data for KEST
getting data for KFFL
getting data for KFOD
getting data for KFSW
getting data for KICL
getting data for KIOW
getting data for KLWD
getting data for KLRJ
getting data for KMCW
getting data for KMIW
getting data for KMUT
getting data for KMXO
getting data for KOLZ
getting data for KORC
getting data for KOTM
getting data for KOXV
getting data for KRDK
getting data for KSDA
getting data for KSHL
getting data for KSPW
getting data for KSLB
getting data for KSUX
getting data for KTNU
getting data for K77M
getting data for KBOI
getting data for KBYI
getting data for KCOE
getting data for KEUL
getting data for KIDA
getting data for KJER
getting data for KLWS
getting data for KMLD
getting data for KMLP
getting data for KMUO
getting data for KMYL
getting data for KP69
getting data for KPIH
getting data for KS14
getting data for KSMN
getting da

getting data for KSZL
getting data for KTBN
getting data for KVIH
getting data for KUNO
getting data for KCDJ
getting data for KCKM
getting data for KHSA
getting data for KOLV
getting data for KHKS
getting data for KHBG
getting data for KUOX
getting data for KBIX
getting data for KCBM
getting data for KGLH
getting data for KGPT
getting data for KGTR
getting data for KGWO
getting data for KHEZ
getting data for KJAN
getting data for KMCB
getting data for KMEI
getting data for KNMM
getting data for KPQL
getting data for KPIB
getting data for KTUP
getting data for KTVR
getting data for K8S0
getting data for K3DU
getting data for KBHK
getting data for K3HT
getting data for K3TH
getting data for KBIL
getting data for KBTM
getting data for KBZN
getting data for KCTB
getting data for KDLN
getting data for KFCA
getting data for KGDV
getting data for KGGW
getting data for KGTF
getting data for KHLN
getting data for KHMM
getting data for KHVR
getting data for KJDN
getting data for KLVM
getting da

In [ ]:
dat